# Training

Welcome to the "_Training_" tutorial of the "_From Zero to Hero_" series. In this part we will present the functionalities offered by the `training` module.

In [ ]:
!pip install git+https://github.com/ContinualAI/avalanche.git

## 💪 The Training Module

The `training` module in _Avalanche_ is build on modularity and its main goals are two:

1. provide a set of standard **continual learning baselines** that can be easily run for comparison;
2. provide the necessary utilities to **create and run your own strategy** as efficiently and easy as possible with building blocks we already prepared for you.

At the moment the `training` module offers two submodule:

* **Strategies**: it containes the collection of pre-implemented baselines you can use for comparisons and base classes to inherit to create your strategy.
* **Plugins**: plugins are modules implementing some specific \(and oftern reusable across strategies\) behaviour you can attach to your own strategy.

## 📈 Strategies

If you want to compare your strategy with other classic continual learning algorithm or baselines, in _Avalanche_ this is as simply as instantiate an object.

In [4]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.strategies import Naive, CWRStar, Replay, GDumb, Cumulative, LwF, GEM, AGEM, EWC

model = SimpleMLP(num_classes=10)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=4, eval_mb_size=100
)

### How to Use Them

Each strategy object offers two main methods `train` and `eval`. Both of them, accept either a _single experience_ \(`Experience`\) or a _list of them_, for maximum flexibility.

In [15]:
import torch
from torchvision import transforms
from torchvision.datasets import MNIST
from avalanche.benchmarks.generators import nc_scenario
from avalanche.models import SimpleMLP

# --- CONFIG
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_experiences = 5
# ---------

# --- TRANSFORMATIONS
train_transform = transforms.Compose([
    transforms.RandomCrop(28, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
# ---------

# --- SCENARIO CREATION
mnist_train = MNIST('./data/mnist', train=True,
                    download=True, transform=train_transform)
mnist_test = MNIST('./data/mnist', train=False,
                   download=True, transform=eval_transform)
scenario = nc_scenario(
    mnist_train, mnist_test, n_experiences, shuffle=True, seed=1234,
    task_labels=False)
# ---------

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=1, eval_mb_size=100,
    device=device)

# TRAINING LOOP
print('Starting experiment...')

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Starting experiment...
Start of experience:  0
Current Classes:  [4, 5]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|████████████████████████████████████████████████████████████████████████████████| 113/113 [00:53<00:00,  2.11it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.3703
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8586
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 25.68it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1873
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9466
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████

-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 26.92it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 8.1148
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 30.08it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp002 = 6.6829
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 28.95it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp003 = 

## 📝Create your Strategy

In _Avalanche_ you can build your own strategy in 2 main ways:

1. **From Scratch Mode**: This is the simplest way to build your own strategy, where you only have to create a python class that implements the main `train` and `eval` methods. However, in this case, all the plugins utilities \(like the evaluation ones that we'll see in the follow up tutorial\) cannot be automatically integrated in your algorithm. You should integrate them "_manually_".
2. **Callbacks Mode**: At this level _Avalanche_ offers a `BaseStrategy` you can inherit from. This strategy offers a simple API and defines the _training and evaluation flows_ i.e. the order of functions to be called when the _train_ and _eval_ methods are triggered. Many of these functions have been already implemented for you in their basic form so you can define a new strategy simply by specializing a few of them. All the evaluation utilities are already integrated in the `BaseStrategy`.

If the standard training and evaluation flows, for some reason, do not fit in your ideal continual learning strategy, you're free to change them as well, as simply as redefining the `train` and `eval` methods in the `BaseStrategy`.

In general, we suggest to work with the _Callbacks Mode_, which offers in our opinion the right level of automation at a reasonable complexity, even for _Avalanche_ "_newbies_". But let's see an example for each of those modalities!

### From Scratch Mode

Let's define our Continual Learning algorithm "_MyStrategy_" as a simple python class:

In [22]:
class MyStrategy():
    """My Basic Strategy"""

    def __init__(self, model, optimizer, criterion):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion

    def train(self, experience):
        # here you can implement your own training loop for each experience (i.e.
        # batch or task).

        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            train_dataset, batch_size=128
        )

        for epoch in range(1):
            for mb in train_data_loader:
                # you magin here...
                pass

    def eval(self, experience):
        # here you can implement your own evaluation loop for each experience
        # (i.e. batch or task).

        eval_dataset = experience.dataset
        t = experience.task_label
        eval_data_loader = DataLoader(
            eval_dataset, batch_size=128
        )

        # eval here

Then, we can use our strategy as we would do for the pre-implemented ones. However, as previously hinted, some evaluation utilities will be note included automatically and should be called by hand. Please refer to the follow-up notebook for more details about what the `evaluation` module can offer.

In [23]:
from avalanche.benchmarks.generators import nc_scenario
from avalanche.models import SimpleMLP
from torch.utils.data import DataLoader

# --- CONFIG
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_experiences = 5
# ---------

# --- TRANSFORMATIONS
train_transform = transforms.Compose([
    transforms.RandomCrop(28, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
# ---------

# --- SCENARIO CREATION
mnist_train = MNIST('./data/mnist', train=True,
                    download=True, transform=train_transform)
mnist_test = MNIST('./data/mnist', train=False,
                   download=True, transform=eval_transform)
scenario = nc_scenario(
    mnist_train, mnist_test, n_experiences, shuffle=True, seed=1234,
    task_labels=False)
# ---------

# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = MyStrategy(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss())

# TRAINING LOOP
print('Starting experiment...')

for experience in scenario.train_stream:
    print("Start of experience ", experience.current_experience)

    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    cl_strategy.eval(scenario.test_stream[experience.current_experience])

Starting experiment...
Start of experience  0
Training completed
Computing accuracy on the whole test set
Start of experience  1
Training completed
Computing accuracy on the whole test set
Start of experience  2
Training completed
Computing accuracy on the whole test set
Start of experience  3
Training completed
Computing accuracy on the whole test set
Start of experience  4
Training completed
Computing accuracy on the whole test set


### Callbacks Mode

At this level _Avalanche_ offers a `BaseStrategy` you can inherit from. This strategy offers a simple API and defines the _training and evaluation flows_ to help you build your own by specializing just a few methods.

**But why do we need inheritance to implement a new continual learning algorithm?**

We noticed that many continual learning strategy follows roughly the same training/evaluation flows and implement the same boilerplate code.

So you it seems natural to define a new strategy just by specializing a few methods. If strategies are defined only "_by difference_" from a basic one, this will **reduce overhead and code duplication**, **improving code readability and prototyping speed**.

**Base Strategy**

As we previously mentioned _Training_ and _Evaluation flows_ are just sequences of functions called within the `BaseStrategy`.

The **standard flows** defined in _Avalanche_ contain the following functions:

```text
train
    before_training
    before_training_exp
    adapt_train_dataset
    make_train_dataloader
    before_training_epoch
        before_training_iteration
            before_forward
            after_forward
            before_backward
            after_backward
        after_training_iteration
        before_update
        after_update
    after_training_epoch
    after_training_exp
    after_training
```

```text
eval
    before_eval
    adapt_eval_dataset
    make_eval_dataloader
    before_eval_exp
        eval_epoch
            before_eval_iteration
            before_eval_forward
            after_eval_forward
            after_eval_iteration
    after_eval_exp
    after_eval
```

The are all the functions contained in the flows as defined by the `BaseStrategy`, the basic class we suggest you to inherit for implementing your continual learning algorithms.

For this class, all the methods \(that you can of course override\) in the flows are implemented for you. So, creating a plain `Replay` algorithm can be done as simply as defining the subclass below:

In [24]:
from avalanche.training.strategies import BaseStrategy


class MyReplay(BaseStrategy):

    def __init__(self, model, optimizer, criterion,
                 mem_size=200,
                 evaluation_protocol=None,
                 train_mb_size=1, train_epochs=1,
                 eval_mb_size=None, device=None,
                 plugins=None):

        super().__init__(
            model, optimizer, criterion, evaluation_protocol,
            train_mb_size=train_mb_size, train_epochs=train_epochs,
            eval_mb_size=eval_mb_size, device=device, plugins=plugins)

        self.mem_size = mem_size
        self.ext_mem = None
        self.it = 0
        self.rm_add = None

    def adapt_train_dataset(self, **kwargs):
        """
        Expands the current training set with datapoint from
        the external memory before training.
        """

        # remember to call the super method
        super().adapt_train_dataset(**kwargs)

        # Additional set of the current batch to be concatenated to the ext.
        # memory at the end of the training
        self.rm_add = None

        # how many patterns to save for next iter
        h = min(self.mem_size // (self.it + 1), len(self.current_data))

        # We recover it using the random_split method and getting rid of the
        # second split.
        self.rm_add, _ = random_split(
            self.current_data, [h, len(self.current_data) - h]
        )

        if self.it > 0:
            # We update the train_dataset concatenating the external memory.
            # We assume the user will shuffle the data when creating the data
            # loader.
            self.current_data = ConcatDataset([self.current_data,
                                               self.ext_mem])

    def after_training_exp(self, **kwargs):
        """ After training we update the external memory with the patterns of
         the current training batch/task. """

        # remember to call the super method
        super().adapt_train_dataset(**kwargs)

        # replace patterns in random memory
        ext_mem = self.ext_mem
        if self.it == 0:
            ext_mem = copy.deepcopy(self.rm_add)
        else:
            _, saved_part = random_split(
                ext_mem, [len(self.rm_add), len(ext_mem) - len(self.rm_add)]
            )
            ext_mem = ConcatDataset([saved_part, self.rm_add])
        self.ext_mem = ext_mem
        self.it += 1

When specializing a method from the parent class, **remember to call "super" one** for the necessarely strategy bookkeeping or to retain the original method behavior \(check the `BaseStrategy` class for more details\).

**Join Training**

When prototyping new continual learning algorithms we often want to understand how the same model would perform when trained on whole stream of data all together. This is often referred to as the "_Cumulative_", "_Joint-training_" or "_Offline_" upper bound.

In Avalanche this can be done with another basic strategy you can extend called `JointTraining`. `JointTraining` follows roughtly the same ideas and API of the `BaseStrategy` but instead of processing experience in a streams iteratively it collapse them into a single, big dataset from which to learn.

It supports both streams with one or multiple tasks:

In [28]:
import torchvision
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.training.strategies import JointTraining

# MNIST dataset urls
prev_mnist_urls = torchvision.datasets.MNIST.resources
new_resources = [
    ('https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz', prev_mnist_urls[0][1]),
    ('https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz', prev_mnist_urls[1][1]),
    ('https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz', prev_mnist_urls[2][1]),
    ('https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz', prev_mnist_urls[3][1])
]
torchvision.datasets.MNIST.resources = new_resources

# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model
model = SimpleMLP(num_classes=10)

# CL Benchmark Creation
perm_mnist = PermutedMNIST(n_experiences=5)
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# Prepare for training & evaluation
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# Joint training strategy
joint_train = JointTraining(
    model, optimizer, criterion, train_mb_size=32, train_epochs=1,
    eval_mb_size=32, device=device)

# train and eval loop
results = []
joint_train.train(train_stream)
results.append(joint_train.eval(test_stream))

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████████████████████████████████████████████████████████████████████████| 1875/1875 [04:23<00:00,  7.10it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2856
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8620
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [00:07<00:00, 39.54it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.2152
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9379
-- Starting eval on experience 1 (Task 1) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [00:07<00:00, 39.65it/s]
> Eval on experience 1 (Task 1) from test stre

### Further Customization

If the standard training and evaluation flows, for some reason, do not fit in your ideal continual learning strategy, you're free to change them as well. You can change the existing flow just by _redefining the train and eval_ methods or any method called in the flow. Everthing is possibile in _Avalanche_! :-\)

In this case we suggest to start by taking a closer look at the inner functioning of the `BaseStrategy` and `JointTraining` classes. We will add more base strategy from which to inherit with different flows as we see they become more needed by the community.

Otherwise, you can just ask us to change the current training and evaluation flows opening a new issue, and we will make sure to help you!

## 🔌 Plugins

As we previously hinted **Plugins** are modules implementing _some specific behaviors you can use to build your strategy more easily_.

The basic idea is to **attach them to the main strategy so that they can augment its behaviors.** In practice this means that in the flow execution, for every method of the main strategy, the plugin\(s\) corresponding methods **will be called first** \(if implemented\).

Under the hoods, the `BaseStrategy`, for each method of the flows, calls the respective methods for each of the plugins attached to it.

An example of Plugin available in _Avalanche_ is the **Evaluation Plugin**. This plugin will handle automatically all metrics computation and logging. A plugin can be added to the `BaseStrategy` simply by passing it to the constructor thought the name parameter `plugins`.

At the moment, in _Avalanche_ are available the following plugins:

In [29]:
from avalanche.training.plugins import ReplayPlugin, \
GDumbPlugin, EvaluationPlugin, CWRStarPlugin, MultiHeadPlugin, LwFPlugin, \
AGEMPlugin, GEMPlugin, EWCPlugin, SynapticIntelligencePlugin

### Create your Plugin

Creating a plugin is rather straightforward in _Avalanche_. You simply need to create a class inheriting from `StrategyPlugin` and implement the callbacks of your choice. This is, for example, the implementation of the replay plugin:

In [30]:
from avalanche.training.plugins import StrategyPlugin

class ReplayPlugin(StrategyPlugin):
    """
    Experience replay plugin.

    Handles an external memory filled with randomly selected
    patterns and implements the "adapt_train_dataset" callback to add them to
    the training set.

    The :mem_size: attribute controls the number of patterns to be stored in
    the external memory. We assume the training set contains at least
    :mem_size: data points.
    """
    def __init__(self, mem_size=200):
        super().__init__()

        self.mem_size = mem_size
        self.ext_mem = None
        self.it = 0
        self.rm_add = None

    def adapt_train_dataset(self, strategy, **kwargs):
        """
        Expands the current training set with datapoint from
        the external memory before training.
        """

        # Additional set of the current batch to be concatenated to the ext.
        # memory at the end of the training
        self.rm_add = None

        # how many patterns to save for next iter
        h = min(self.mem_size // (self.it + 1), len(strategy.current_data))

        # We recover it using the random_split method and getting rid of the
        # second split.
        self.rm_add, _ = random_split(
            strategy.current_data, [h, len(strategy.current_data) - h]
        )

        if self.it > 0:
            # We update the train_dataset concatenating the external memory.
            # We assume the user will shuffle the data when creating the data
            # loader.
            strategy.current_data = ConcatDataset([strategy.current_data,
                                                   self.ext_mem])

    def after_training(self, strategy, **kwargs):
        """ After training we update the external memory with the patterns of
         the current training batch/task. """

        # replace patterns in random memory
        ext_mem = self.ext_mem
        if self.it == 0:
            ext_mem = copy.deepcopy(self.rm_add)
        else:
            _, saved_part = random_split(
                ext_mem, [len(self.rm_add), len(ext_mem)-len(self.rm_add)]
            )
            ext_mem = ConcatDataset([saved_part, self.rm_add])
        self.ext_mem = ext_mem
        self.it += 1


__all__ = ['ReplayPlugin']

Easy, isn't it? :-\)

In general we recommend to _implement a Strategy via plugins_, if possible. This will allow other people to use them and facilitate interoperability among different approaches.

For example, we could have implemented a replay strategy as a standalone child class of the `BaseStrategy`. However, creating a plugin to be used in conjunction with the `Naive` strategy is much better, since _experience replay_ is part of many different continual learning strategies:

In [31]:
from avalanche.training.strategies import Naive

cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=100, train_epochs=4, eval_mb_size=100,
    plugins=[ReplayPlugin(mem_size=10000)]
)

This completes the "_Training_" chapter for the "_From Zero to Hero_" series. We hope you enjoyed it!

## 🤝 Run it on Google Colab

You can run _this chapter_ and play with it on Google Colaboratory: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContinualAI/colab/blob/master/notebooks/avalanche/3.-training.ipynb)